In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import cv2
from torch.utils.data import DataLoader
import glob

In [2]:
batch_size = 64
num_classes = 2
learning_rate = 0.001
num_epochs = 20
PATH = '/content/drive/MyDrive/Projects/simple cnn/'

In [3]:
class ShiriniDataset(Dataset):
    def __init__(self, data_path,  transform=None):
        self.data_path = data_path
        self.classes = glob.glob(data_path+'/*')
        self.transform = transform
        self.image_paths = []
        for c in self.classes:
          self.image_paths += glob.glob(c+'/*')
        self.label_map = {'bersagh': 0, 'Khamee': 1}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.resize(image, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label = image_filepath.split('/')[-2]
        tensor_label = torch.tensor(self.label_map[label])

        if self.transform:
            image = self.transform(image)

        return image, tensor_label

In [4]:
transform = transforms.Compose([
  transforms.ToPILImage(),
  transforms.RandomResizedCrop(128),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor()])

In [5]:
train_dataset = ShiriniDataset(PATH+"Shirini/Train", transform)

test_dataset = ShiriniDataset(PATH+"Shirini/Test", transform)

train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True)

test_loader = DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False)

In [ ]:
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(53824, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)

        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)

        out = out.reshape(out.size(0), -1)

        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [ ]:
model = ConvNeuralNet(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    weight_decay = 0.005)
total_step = len(train_loader)

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))





Epoch [1/20], Loss: 1.9707
Epoch [2/20], Loss: 1.6641
Epoch [3/20], Loss: 0.3528
Epoch [4/20], Loss: 0.3644
Epoch [5/20], Loss: 0.3091
Epoch [6/20], Loss: 0.2755
Epoch [7/20], Loss: 0.4070
Epoch [8/20], Loss: 0.1948
Epoch [9/20], Loss: 0.5214
Epoch [10/20], Loss: 0.4506
Epoch [11/20], Loss: 0.2411
Epoch [12/20], Loss: 0.3268
Epoch [13/20], Loss: 0.3496
Epoch [14/20], Loss: 0.5864
Epoch [15/20], Loss: 0.2489
Epoch [16/20], Loss: 0.2091
Epoch [17/20], Loss: 0.3125
Epoch [18/20], Loss: 0.2253
Epoch [19/20], Loss: 0.3318
Epoch [20/20], Loss: 0.3048


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network : {} %'.format(100 * correct / total))




Accuracy of the network : 92.5 %


In [ ]:
torch.save(model.state_dict(), PATH+'ShiriniModel.pth')